In [27]:
import cv2
import os, glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rembg import remove

In [28]:
# # 데이터 관련 경로
folder_path = '../../opencv_img/' # 사진 파일들이 있는 폴더 경로
file_path_list = os.listdir(folder_path)

# # new_path = r"C:\Users\KDP-040\Desktop\bottle\glass_bottle"
# cnt=1
# for filename in os.listdir(folder_path):
#     if filename.endswith(".jpg"):
#         date_part = filename.split("_")[0]  # 날짜 부분 추출
#         new_filename = f"paper_{cnt:<2}.jpg"  # 새로운 파일명 생성
        
#         old_path = os.path.join(folder_path, filename)
#         new_path = os.path.join(folder_path, new_filename)
        
#         os.rename(old_path, new_path)  # 파일 이름 변경
#         cnt+=1

In [29]:
def imgRotate(arr):
    arr = arr.reshape((150, 150))
    image_list = []

    height, width = arr.shape[:2]
    # 회전 변환 행렬 반환
    for degree in range(90, 360, 90):
        matrix = cv2.getRotationMatrix2D((width/2, height/2), degree, 1)

        # 회전 변환행렬에 따른 회전 이미지 반환
        rotated_image = cv2.warpAffine(arr, matrix, (width, height))
        image_list.append(rotated_image.flatten())
    final_img_array = np.array(image_list)
    return final_img_array 

In [30]:
final_df = pd.DataFrame([])

for filename in os.listdir(folder_path):
    if filename.endswith(".jpg"):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        resized_img = cv2.resize(img, (150, 150))

        df = pd.DataFrame(resized_img.flatten()).T  # 이미지 배열을 1차원으로 변환하고 행 벡터로 전치
        final_df = pd.concat([final_df, df]) 

        # rotate_df = pd.DataFrame(imgRotate(resized_img))
        # final_df = pd.concat([final_df, df]) 
final_df.reset_index(drop=True, inplace=True)

In [31]:
final_df.shape

(851, 22500)

In [32]:
final_df.tail()

,0,1,2,3,4,5,6,7,8,9,...,22490,22491,22492,22493,22494,22495,22496,22497,22498,22499
846,202,201,199,200,202,202,196,190,203,200,...,182,183,192,186,191,189,190,200,194,194
847,198,198,198,197,200,200,197,202,198,203,...,183,186,183,184,183,188,189,193,194,189
848,196,194,193,198,197,198,194,199,201,196,...,181,182,177,181,186,184,184,189,185,187
849,193,194,192,194,194,192,193,194,194,194,...,169,170,181,176,177,178,171,183,177,175
850,193,193,192,191,190,192,194,196,194,193,...,160,166,174,166,173,174,172,181,175,168


In [33]:
rotated_images_df = pd.DataFrame([])

for index, row in final_df.iterrows():
    row_array = row.values  # 행 데이터를 numpy 배열로 변환
    rotated_images= imgRotate(row_array)
    
    for r in rotated_images:
        rotate_df=pd.DataFrame(r).T
        rotated_images_df=pd.concat([rotated_images_df ,rotate_df])  
    
rotated_images_df.shape

(2553, 22500)

In [34]:
# 확인용

# rotate_test = rotated_images_df.iloc[0, :].to_numpy().reshape(150, 150, 1)
# cv2.imshow('rotate_test', rotate_test)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [35]:
blurred_images_df = pd.DataFrame([])

for index, row in final_df.iterrows():
    row_array = row.values  # 행 데이터를 numpy 배열로 변환
    image = row_array.reshape((150,150))
    blurred_image = cv2.GaussianBlur(image,(3 ,3),5)
    
    blur_df = pd.DataFrame(blurred_image.flatten()).T 
    blurred_images_df = pd.concat([blurred_images_df, blur_df], ignore_index=True)  
    
blurred_images_df.shape

(851, 22500)

In [36]:
# 확인용

# blurred_test = blurred_images_df.iloc[0, :].to_numpy().reshape(150, 150, 1)
# cv2.imshow('blurred_test', blurred_test)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [37]:
sharp_images_df = pd.DataFrame([])
kernel = np.array([[-1, -1, -1], 
                   [-1, 9,-1], 
                   [-1, -1, -1]])

for index,row in final_df.iterrows():
    row_array=row.values 
    image=row_array.reshape((150 ,150)) 

    # 필터 적용하기
    sharp_image=cv2.filter2D(image,-1,kernel) 

    sharp_df=pd.DataFrame(sharp_image.flatten()).T 
    sharp_images_df=pd.concat([sharp_images_df, sharp_df], ignore_index=True)
sharp_images_df.shape

(851, 22500)

In [38]:
# 확인용

# sharped_test = sharp_images_df.iloc[0, :].to_numpy().reshape(150, 150, 1)
# cv2.imshow('sharped_test', sharped_test)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [39]:
final_ = pd.concat([final_df, rotated_images_df, blurred_images_df, sharp_images_df], ignore_index=True)

In [40]:
final_['label'] = 'paper'

In [41]:
final_.shape

(5106, 22501)

In [42]:
final_.head(10)

,0,1,2,3,4,5,6,7,8,9,...,22491,22492,22493,22494,22495,22496,22497,22498,22499,label
0,192,196,196,186,182,187,189,189,192,195,...,196,186,193,194,190,186,179,192,177,paper
1,195,194,193,202,189,201,194,193,195,192,...,191,187,191,185,182,180,185,188,182,paper
2,201,203,202,201,197,203,203,201,201,199,...,188,200,190,186,196,204,193,203,198,paper
3,204,193,201,202,203,197,203,205,207,197,...,189,191,192,193,190,189,191,187,189,paper
4,198,207,203,191,205,204,199,195,203,205,...,186,188,189,194,193,190,194,190,190,paper
5,195,202,204,197,204,204,206,196,201,208,...,187,181,185,187,188,184,185,186,188,paper
6,204,202,201,207,202,202,197,204,204,197,...,185,183,188,184,184,183,185,189,184,paper
7,197,202,207,191,204,208,202,197,201,207,...,194,184,174,185,187,174,179,187,194,paper
8,203,194,204,201,200,196,201,205,199,198,...,175,186,185,182,185,184,181,179,182,paper
9,194,204,203,199,194,203,205,198,199,200,...,175,183,182,177,177,182,180,179,186,paper


In [43]:
final_.head()

,0,1,2,3,4,5,6,7,8,9,...,22491,22492,22493,22494,22495,22496,22497,22498,22499,label
0,192,196,196,186,182,187,189,189,192,195,...,196,186,193,194,190,186,179,192,177,paper
1,195,194,193,202,189,201,194,193,195,192,...,191,187,191,185,182,180,185,188,182,paper
2,201,203,202,201,197,203,203,201,201,199,...,188,200,190,186,196,204,193,203,198,paper
3,204,193,201,202,203,197,203,205,207,197,...,189,191,192,193,190,189,191,187,189,paper
4,198,207,203,191,205,204,199,195,203,205,...,186,188,189,194,193,190,194,190,190,paper


In [44]:
final_.tail()

,0,1,2,3,4,5,6,7,8,9,...,22491,22492,22493,22494,22495,22496,22497,22498,22499,label
5101,220,220,188,207,212,224,166,109,233,190,...,155,235,155,204,170,163,254,186,200,paper
5102,196,194,195,183,213,219,195,245,183,227,...,218,175,198,157,202,194,228,230,185,paper
5103,222,199,179,214,191,209,165,216,226,181,...,212,146,178,205,174,169,216,161,183,paper
5104,189,207,186,210,212,193,199,203,200,198,...,132,227,174,189,214,146,255,215,177,paper
5105,207,206,194,185,173,190,206,224,203,186,...,162,228,129,191,191,145,228,182,100,paper


In [45]:
# 최종 확인용 (원본)

origin_idx = final_df.shape[0] -1 # 원본 A
rotate_idx = origin_idx + (rotated_images_df.shape[0]-1) # 원본 A의 회전 인덱스
blurr_idx = rotate_idx + (blurred_images_df.shape[0]-1) # 원본 A의 블러 인덱스
sharpen_idx = blurr_idx + (sharp_images_df.shape[0]-1) # 원본 A의 샤픈 인덱스

original = final_.iloc[origin_idx, :-1].to_numpy().reshape(150, 150, 1)
original = original.astype(np.uint8) 
cv2.imshow('original', original)
cv2.waitKey()
cv2.destroyAllWindows()

In [46]:
# 최종 확인용 (회전)

rotate = final_.iloc[rotate_idx, :-1].to_numpy().reshape(150, 150, 1)
rotate = rotate.astype(np.uint8) 
cv2.imshow('rotate', rotate)
cv2.waitKey()
cv2.destroyAllWindows()

In [47]:
# 최종 확인용 (블러)

blurr = final_.iloc[blurr_idx, :-1].to_numpy().reshape(150, 150, 1)
blurr = blurr.astype(np.uint8) 

cv2.imshow('blurr', blurr)
cv2.waitKey()
cv2.destroyAllWindows()

In [48]:
# 최종 확인용 (샤프)

sharp = final_.iloc[sharpen_idx, :-1].to_numpy().reshape(150, 150, 1)
sharp = sharp.astype(np.uint8) 
cv2.imshow('sharp', sharp)
cv2.waitKey()
cv2.destroyAllWindows()

In [49]:
final_.to_csv('paper_all.csv', index=False)